In [1]:
import os

from option.option import parse
from data import create_dataset, create_dataloader
from model import define_C
from solver import create_solver

In [2]:
opt = parse('./option/train/train_dense161.json')

export CUDA_VISIBLE_DEVICES=1,2


In [3]:
for k, v in opt['path'].items():
    if not os.path.exists(v):
        os.makedirs(v)

In [4]:
solver = create_solver(opt)

===> Network Summary

Network structure: [DataParallel - densenet161], with parameters: [26,516,180]
===> Solver Initialized : [OCSolver] || Use GPU : [True]
optimizer_C:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.0001
    lr: 0.0001
    weight_decay: 0.0001
)
lr_scheduler milestones: [10, 15, 18]   gamma: 0.500000


In [5]:
for phase, dataset_opt in opt['datasets'].items():
    if phase == 'train':
        train_set = create_dataset(dataset_opt)
        train_loader = create_dataloader(train_set, dataset_opt)
    elif phase == 'val':
        val_set = create_dataset(dataset_opt)
        val_loader = create_dataloader(val_set, dataset_opt)

In [6]:
it = iter(train_loader)

In [ ]:
sample = next(it)
solver.feed_data(sample)
solver.train_step()